pyltp 安装指南：

使用whl轮子安装:  
链接：https://pan.baidu.com/s/1Jbw6IjVbb-URZCHvh3FNDQ  
提取码：i3le  

pyltp数据包安装:  
pyltp要能实现分词、词性标注、命名实体识别，还需要安装数据包以及模型。请确保下载的模型版本与当前版本的 pyltp 对应，否则会导致程序无法正确加载模型。  
模型下载地址：https://pan.baidu.com/share/link?shareid=1988562907&uk=2738088569#list/path=%2F  

参考资料：
- https://gitee.com/v_wanglei/entity_relation_extraction/blob/master/%E5%AE%9E%E4%BD%93%E5%85%B3%E7%B3%BB%E6%8A%BD%E5%8F%96.md
- https://www.docin.com/p-1715877509.html

导包：

In [1]:
import sys
import os
 
from pyltp import Segmentor, Postagger, Parser, NamedEntityRecognizer

In [13]:
MODELDIR = "ltp_data/" # 注意斜杠方向
segmentor = Segmentor(os.path.join(MODELDIR, "cws.model")) # 分词器
#segmentor.load(os.path.join(MODELDIR, "cws.model"))

postagger = Postagger(os.path.join(MODELDIR, "pos.model")) # 词性标注
#postagger.load(os.path.join(MODELDIR, "pos.model"))

recognizer = NamedEntityRecognizer(os.path.join(MODELDIR, "ner.model")) # 命名实体识别
#recognizer.load(os.path.join(MODELDIR, "ner.model"))

parser = Parser(os.path.join(MODELDIR, "parser.model")) # 依存句法分析
#parser.load(os.path.join(MODELDIR, "parser.model"))

输入待分析的文本：

In [69]:
#sentence = "父亲此后前往哈佛大学求学"
#sentence = "德国国防部正在制订行动计划的细节。"
sentence = "德国政府于26日原则上决定参与有关军事行动"

分词

In [70]:
WORDS = segmentor.segment(sentence)
print("\t".join(WORDS))

德国	政府	于	26日	原则	上	决定	参与	有关	军事	行动


词性标注

In [71]:
POSTAGS = postagger.postag(WORDS)
print("\t".join("%s" % postag for postag in POSTAGS))
len(WORDS)

ns	n	p	nt	n	nd	v	v	v	n	v


11

命名实体识别

In [72]:
NETAGS = recognizer.recognize(WORDS, POSTAGS)
print("\t".join("%s" % netag for netag in NETAGS))

B-Ni	E-Ni	O	O	O	O	O	O	O	O	O


依存句法分析

In [73]:
ARCS = parser.parse(WORDS, POSTAGS) 
print("\t".join("%d:%s" % (arc.head, arc.relation) for arc in ARCS))
len(ARCS)

2:ATT	7:SBV	7:ADV	5:ATT	6:ATT	3:POB	0:HED	7:VOB	11:ATT	11:ATT	8:VOB


11

德国(1)   政府	   于	  26日	 原则	    上	   决定	  参与	 有关	   军事    行动  
2:ATT	7:SBV	7:ADV	5:ATT	6:ATT	3:POB	0:HED	7:VOB	11:ATT	11:ATT	8:VOB

为句子中的每个词语维护一个保存句法依存儿子节点的字典：

In [74]:
def build_parse_child(arcs):
    """
    Args:
        arcs: 句法依存列表
    """
    words_children = []
    for index in range(len(arcs)):
        word_children = {}
        for arc_index,arc in enumerate(arcs):
            if arc.head == index+1: # 默认 head 为 0 
                if arc.relation in word_children:
                    word_children[arc.relation].append(arc_index)
                else:
                    word_children[arc.relation] = [arc_index]
        words_children.append(word_children)
    return words_children

In [75]:
WORDS_CHILDREN = build_parse_child(ARCS)
WORDS_CHILDREN # 每个元素为对应的单词及其儿子节点（加上相关关系）

[{},
 {'ATT': [0]},
 {'POB': [5]},
 {},
 {'ATT': [3]},
 {'ATT': [4]},
 {'SBV': [1], 'ADV': [2], 'VOB': [7]},
 {'VOB': [10]},
 {},
 {},
 {'ATT': [8, 9]}]

补全识别的部分实体：

比如：行动(VOB) -> 有关(ATT)   军事(ATT)   行动(VOB)

In [76]:
def get_complete_ner(word_index):
    word_children = WORDS_CHILDREN[word_index]
    prefix = "" # 前缀
    if "ATT" in word_children:
        for i in range(len(word_children["ATT"])):
            prefix += get_complete_ner(word_children["ATT"][i])
    postfix = "" # 后缀
    if POSTAGS[word_index] == "v":
        if "VOB" in word_children:
            postfix += get_complete_ner(word_children["VOB"][0])
        if "SBV" in word_children:
            postfix += get_complete_ner(word_children["SBV"][0]) + prefix
    return prefix + WORDS[word_index] + postfix

对于给定的句子进行事实三元组抽取:

In [83]:
def extract_triple():
    for index,pos in enumerate(POSTAGS):
        # 抽取以谓词为中心的事实三元组
        if pos == "v": # 如果为动词
            word_children = WORDS_CHILDREN[index] # 取其对应的儿子节点
            # 主谓宾，例：我送她一束花
            if "SBV" in word_children and "VOB" in word_children:
                e1 = get_complete_ner(word_children["SBV"][0])
                r = WORDS[index]
                e2 = get_complete_ner(word_children["VOB"][0])
                print("主语谓语宾语关系\t(%s, %s, %s)\n" % (e1, r, e2))
            # 定语后置，动宾关系，例：位于火奴鲁鲁的大型私立学校
            if ARCS[index].relation == "ATT":
                if "VOB" in word_children:
                    e1 = get_complete_ner(ARCS[index].head-1)
                    r = WORDS[index]
                    e2 = get_complete_ner(word_children["VOB"][0])
                    temp = r + e2
                    if temp == e1[:len(temp)]:
                        e1 = e1[len(temp):]
                    if temp not in e1:
                        print("定语后置动宾关系\t(%s, %s, %s)\n" % (e1, r, e2))
            # 含有介宾关系的主谓动补关系，例：哈立德居住在土耳其
            if "SBV" in word_children and "CMP" in word_children:
                e1 = get_complete_ner(word_children["SBV"][0])
                cmp_index = word_children["CMP"][0]
                r = WORDS[index] + WORDS[cmp_index]
                if "POB" in WORDS_CHILDREN[cmp_index]:
                    e2 = get_complete_ner(WORDS_CHILDREN[cmp_index]["POB"][0])
                    print("介宾关系主谓动补\t(%s, %s, %s)\n" % (e1, r, e2))
        

批量实验：

In [84]:
with open("/usr/local/codeData/KG-July/4/input.txt","r",encoding="utf-8") as f:
    lines = f.readlines()
for s in lines:
    print("原句",s.strip())
    WORDS = segmentor.segment(s.strip())
    POSTAGS = postagger.postag(WORDS)
    ARCS = parser.parse(WORDS, POSTAGS) 
    WORDS_CHILDREN = build_parse_child(ARCS)
    extract_triple()

原句 贝拉克·侯赛因·奥巴马的身世复杂，1961年8月4日出生在美国夏威夷州檀香山市，父亲是来自肯尼亚的留学生，母亲是堪萨斯州白人。
主语谓语宾语关系	(父亲, 是, 来自肯尼亚留学生)

定语后置动宾关系	(留学生, 来自, 肯尼亚)

主语谓语宾语关系	(母亲, 是, 堪萨斯州白人)

原句 他们二人在就读夏威夷大学期间相识。
定语后置动宾关系	(期间, 就读, 夏威夷大学)

原句 由于父亲此后前往哈佛大学求学，奥巴马从小由母亲抚养，奥巴马两岁多时，他的父母婚姻破裂。
主语谓语宾语关系	(父亲, 前往, 哈佛大学)

原句 6岁时，奥巴马随母亲和继父前往印度尼西亚首都雅加达生活，并在当地的一所小学就读了两年。
主语谓语宾语关系	(奥巴马, 前往, 印度尼西亚首都雅加达)

原句 四年后他的一家又回到夏威夷，与外祖父母住在一起，从五年级起，他就读于位于火奴鲁鲁的大型私立学校——普纳荷学校（中华民国第一任临时大总统孙文曾于此校就读）至12年级，于1979年毕业，若干年后母亲与继父离婚，他便随母迁居美国本土。
主语谓语宾语关系	(他一家, 回到, 夏威夷)

介宾关系主谓动补	(他, 就读于, 就读位于火奴鲁鲁大型私立学校普纳荷学校中华民国第一任临时大总统孙文)

定语后置动宾关系	(大型私立学校, 位于, 火奴鲁鲁)

主语谓语宾语关系	(他, 迁居, 美国本土)

原句 青年时期，奥巴马因为自己的多种族背景，很难取得社会认同，十分自卑，他过了一段荒唐的日子，做了很多愚蠢的事，比如逃学、吸毒、泡妞等，成了一个不折不扣的“迷途叛逆少年”。
主语谓语宾语关系	(奥巴马, 取得, 社会认同)

主语谓语宾语关系	(他, 过, 一段荒唐日子)

原句 十几岁的他成了一个瘾君子，曾以吸食大麻和可卡因来“将‘我是谁’的问题挤出脑袋”。
主语谓语宾语关系	(十几岁他, 成, 一个瘾君子)

主语谓语宾语关系	(来, 是, 谁)

主语谓语宾语关系	(问题, 挤出, 脑袋)

原句 给青年的他带来深刻影响的不是他的父母亲，而是他的外祖父斯坦利·埃默·邓汉姆和外祖母斯坦利·安·邓汉姆，著名黑人诗人、记者和美国左翼活动家法兰克·米歇尔·戴维斯也是深刻影响青年奥巴马的人物，19世纪60年代戴维斯就成为奥巴马家里的常客。
主语谓语宾语关系	(带来深刻影响, 是, 他父母亲)

主语谓语宾